In [1]:
import math
import random
import torch

In [2]:
from graphviz import Digraph

def trace(root):
    # Build a set of all nodes and edges in a graph and input is an object
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={"rankdir": "LR"}) # LR = left to Right
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        # for any value in the graph, create a rectangular ('record') node for it
        dot.node(name=uid, label= "{ %s | data %.4f | grad %.4f }" %(n.label, n.data, n.grad, ), shape="record")
        if n._op:
            #if this value is a result of some operation, create an op node for it
            dot.node(name=uid + n._op, label=n._op)
            # Connect this node to it
            dot.edge(uid + n._op, uid)
    for n1, n2 in edges:
        # Connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [3]:
class Value:
    def __init__(self, data, _children=(), _op="", label=""):
        self.data = data
        self.grad = 0   # default value of grad is zero during first epoch
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
        
    def __repr__(self):
        return f"Value(data={self.data})"
        
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), "+")
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out
        
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), "*")
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)) # Only supports int and float
        out = Value(self.data**other, (self,), f"**{other}")
        def _backward():
            self.grad += other * (self.data ** (other-1)) * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self*-1

    def __sub__(self, other):
        return self + (-other)

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * (other**-1)
        
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), "tanh")
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = self.data 
        out = Value(math.exp(x), (self, ), "exp")
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out        

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()        

In [4]:
x1 = torch.tensor([2.0]).double();  x1.requires_grad = True;
x2 = torch.tensor([0.0]).double();  x2.requires_grad = True;
w1 = torch.tensor([-3.0]).double(); w1.requires_grad = True;
w2 = torch.tensor([1.0]).double();  w2.requires_grad = True;
b = torch.tensor([6.8813735870]).double(); b.requires_grad = True;
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)
print("Data Value at node 'o': ", o.data.item())

print("============")
o.backward()
print("x2", x2.grad.item())
print("w2", w2.grad.item())
print("x1", x1.grad.item())
print("w1", w1.grad.item())

Data Value at node 'o':  0.7071066904050358
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [5]:
class Neuron:
    # This will do pre-activation (wi*xi) + b and activation function tanh()
    def __init__(self, nin):     # nin is number of inputs
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]  # len(nin) objects will be created
        self.b = Value(random.uniform(-1,1))                        # one Value object will be created

    def __call__(self, x):
        # wi*xi + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)  # Important 
        out = act.tanh()
        return out        
        
class Layer:    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

class MLP:
    # Multi Layer Perceptron
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
                
    
x = [2.0, 3.0, -1.0]
n = MLP(3,[4,4,1])
n(x)        

Value(data=0.5819628024249682)

In [6]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],    
]
ys = [1.0, -1.0, -1.0, 1.0]  # desired targets
ypred = [n(x) for x in xs]
ypred

[Value(data=0.5819628024249682),
 Value(data=0.8438234257403808),
 Value(data=0.5124041831996848),
 Value(data=0.7773542536985493)]

In [9]:
loss = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
loss

[Value(data=0.17475509855638616),
 Value(data=3.3996848253089933),
 Value(data=2.287366413359906),
 Value(data=0.049571128346129964)]